In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
import torchreid
from deep_sort_realtime.deepsort_tracker import DeepSort
from sklearn.metrics.pairwise import cosine_similarity
import torch
import time
import os

# --- Initialize YOLOv11 model for detection ---
yolo_model = YOLO("yolo11n.pt")

# --- Initialize Torchreid feature extractor ---
extractor = torchreid.utils.FeatureExtractor(
    model_name='osnet_x1_0',
    device='cpu'
)

# --- Initialize DeepSORT tracker ---
tracker = DeepSort(max_age=30, n_init=3)

# --- Setup for FPS display ---
prev_time = 0

# --- In-memory gallery for face embeddings and images ---
gallery = {}
next_person_id = 0
person_id_map = {}

# --- Preset named faces (can store multiple photos per person) ---
preset_faces = {
    "Nishan": [
        "data/nishan.png",
        "data/nishan1.png",
        "data/nishan2.png"
    ],
    
    "Manish": [
        "data/manish.png",
        "data/manish1.png"
    ]
}

# --- Load and store preset faces into gallery ---
for name, paths in preset_faces.items():
    for path in paths:
        img = cv2.imread(path)
        if img is None:
            print(f"[WARN] Could not load image: {path}")
            continue

        resized = cv2.resize(img, (128, 256))
        feature = extractor([resized])[0].cpu()

        if name not in gallery:
            gallery[name] = []

        gallery[name].append({
            'embedding': feature,
            'face': resized,
            'locked': True  # Mark as locked to prevent overwriting
        })

    print(f"[INFO] Added {len(gallery[name])} image(s) for {name} to gallery.")

# --- Utility: Image quality checks ---
def is_better_image(new_img, old_img):
    def sharpness(img):
        return cv2.Laplacian(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), cv2.CV_64F).var()
    def area(img):
        return img.shape[0] * img.shape[1]
    return sharpness(new_img) + area(new_img) > sharpness(old_img) + area(old_img)

def is_good_enough(image, sharp_thresh=100.0, brightness_thresh=(60, 200)):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sharpness = cv2.Laplacian(gray, cv2.CV_64F).var()
    mean_brightness = np.mean(gray)
    return sharpness > sharp_thresh and brightness_thresh[0] < mean_brightness < brightness_thresh[1]

# --- Start webcam feed ---
cap = cv2.VideoCapture(0)
min_conf = 0.5

while True:
    detections = []
    check, frame = cap.read()
    if not check:
        break

    # --- Calculate FPS ---
    cur_time = time.time()
    fps = round(1 / (cur_time - prev_time), 2)
    prev_time = cur_time

    # --- YOLO person detection ---
    result = yolo_model(frame, verbose=False)[0]
    for box in result.boxes:
        if int(box.cls[0]) != 0 or float(box.conf[0]) < min_conf:
            continue
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = float(box.conf[0])
        detections.append(([x1, y1, x2 - x1, y2 - y1], conf, 'person'))

    # --- Track people using DeepSORT ---
    tracks = tracker.update_tracks(detections, frame=frame)
    
    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        l, t, r, b = map(int, track.to_ltrb())
        cropped = frame[t:b, l:r]
        if cropped.size == 0:
            continue

        resized = cv2.resize(cropped, (128, 256))
        feature = extractor([resized])[0].cpu()

        # --- Assign ID or Name ---
        if track_id not in person_id_map:
            best_match_id = None
            best_similarity_score = 0.0

            # Compare with all known gallery faces
            for person_name, faces in gallery.items():
                for face_data in faces:
                    score = cosine_similarity(
                        feature.unsqueeze(0).numpy(),
                        face_data['embedding'].unsqueeze(0).numpy()
                    )[0][0]
                    if score > best_similarity_score:
                        best_similarity_score = score
                        best_match_id = person_name

            # If match is strong enough
            if best_similarity_score > 0.6:
                person_id_map[track_id] = best_match_id
            else:
                # Assign numeric ID
                current_id = str(next_person_id)
                next_person_id += 1
                gallery[current_id] = [{
                    'embedding': feature,
                    'face': resized,
                    'locked': is_good_enough(resized)
                }]
                print(f"[INFO] Stored {'GOOD' if gallery[current_id][0]['locked'] else 'TEMP'} face for new person {current_id}")
                person_id_map[track_id] = current_id

        else:
            # If already known track, check for better image
            pid = person_id_map[track_id]
            faces = gallery[pid]
            if all(not f['locked'] for f in faces):
                current_best = faces[0]
                if is_good_enough(resized) and is_better_image(resized, current_best['face']):
                    gallery[pid][0] = {
                        'embedding': feature,
                        'face': resized,
                        'locked': True
                    }
                    print(f"[INFO] Updated {pid} with better GOOD image")

        # --- Draw bounding box and label ---
        pid = person_id_map[track_id]
        label = pid if not pid.isdigit() else f"Person {pid}"
        cv2.rectangle(frame, (l, t), (r, b), (0, 255, 0), 2)
        cv2.putText(frame, label, (l, t - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # --- Show FPS ---
    cv2.putText(frame, f"FPS: {fps}", (20, 50), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 100, 255), 2)

    # --- Display the frame ---
    cv2.imshow("ReID + Tracking", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# --- Cleanup ---
cap.release()
cv2.destroyAllWindows()


In [ ]:
import matplotlib.pyplot as plt
import cv2

# Collect all stored face images from gallery
saved_faces = []
face_labels = []

for person_id, face_entries in gallery.items():
    for idx, entry in enumerate(face_entries):
        saved_faces.append(entry['face'])
        label = person_id if len(face_entries) == 1 else f"{person_id} #{idx+1}"
        face_labels.append(label)

n = len(saved_faces)

if n == 0:
    print("No faces saved yet.")
else:
    # Grid layout: 5 images per row
    cols = min(n, 5)
    rows = (n + cols - 1) // cols

    plt.figure(figsize=(3 * cols, 4 * rows))

    for i, face in enumerate(saved_faces):
        plt.subplot(rows, cols, i + 1)
        plt.imshow(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))
        plt.title(face_labels[i])
        plt.axis('off')

    plt.tight_layout()
    plt.show()
